# --- Day 6: Guard Gallivant ---

The Historians use their fancy device again, this time to whisk you all away to the North Pole prototype suit manufacturing lab... in the year 1518! It turns out that having direct access to history is very convenient for a group of historians.

You still have to be careful of time paradoxes, and so it will be important to avoid anyone from 1518 while The Historians search for the Chief. Unfortunately, a single guard is patrolling this part of the lab.

Maybe you can work out where the guard will go ahead of time so that The Historians can search safely?

You start by making a map (your puzzle input) of the situation. For example:

....#.....

.........#

..........

..#.......

.......#..

..........

.#..^.....

........#.

#.........

......#...

The map shows the current position of the guard with ^ (to indicate the guard is currently facing up from the perspective of the map). Any obstructions - crates, desks, alchemical reactors, etc. - are shown as #.

Lab guards in 1518 follow a very strict patrol protocol which involves repeatedly following these steps:
- If there is something directly in front of you, turn right 90 degrees.
- Otherwise, take a step forward.

Following the above protocol, the guard moves up several times until she reaches an obstacle (in this case, a pile of failed suit prototypes):

....#.....

....^....#

..........

..#.......

.......#..

..........

.#........

........#.

#.........

......#...

Because there is now an obstacle in front of the guard, she turns right before continuing straight in her new facing direction:

....#.....

........>#

..........

..#.......

.......#..

..........

.#........

........#.

#.........

......#...

Reaching another obstacle (a spool of several very long polymers), she turns right again and continues downward:

....#.....

.........#

..........

..#.......

.......#..

..........

.#......v.

........#.

#.........

......#...

This process continues for a while, but the guard eventually leaves the mapped area (after walking past a tank of universal solvent):

....#.....

.........#

..........

..#.......

.......#..

..........

.#........

........#.

#.........

......#v..

By predicting the guard's route, you can determine which specific positions in the lab will be in the patrol path. Including the guard's starting position, the positions visited by the guard before leaving the area are marked with an X:

....#.....

....XXXXX#

....X...X.

..#.X...X.

..XXXXX#X.

..X.X.X.X.

.#XXXXXXX.

.XXXXXXX#.

#XXXXXXX..

......#X..

In this example, the guard will visit 41 distinct positions on your map.

Predict the path of the guard. How many distinct positions will the guard visit before leaving the mapped area?

In [33]:
class Guard:
    change_dir_map = {"u": "r", "r": "d", "d": "l", "l": "u"}
    dir_vector = {"u": [-1, 0], "r": [0, 1], "d": [1, 0], "l": [0, -1]}

    def __init__(self, start_coordinates: list[int]):
        self.init_position = start_coordinates.copy()
        self.restart_loop_tracker()
        self.main_path = set()
        self.positions_seen = 1
        self.possible_move = None
        self.in_bounds = True

    def change_dir(self):
        self.dir = self.change_dir_map[self.dir]

    def find_next_move(self):
        self.possible_move = [self.position[x] + self.dir_vector[self.dir][x] for x in range(2)]
        return self.possible_move

    def accept_move(self, next_tile: str, part_1: bool = True):
        self.position = self.possible_move
        if part_1:
            self._count_if_new(next_tile == ".")
            self.track_main_path(tuple(self.position))
        else:
            return self.track_potential_loop_path(tuple(self.position))

    def _count_if_new(self, new: bool):
        self.positions_seen += new

    def restart_loop_tracker(self):
        self.position = self.init_position.copy()
        self.dir = "u"
        self.position_visits = set([(self.position[0], self.position[1], self.dir)])
        self.in_bounds = True

    def track_main_path(self, coordinates: tuple[int]):
        if coordinates != tuple(self.init_position):
            self.main_path.add(coordinates)

    def track_potential_loop_path(self, coordinates: tuple[int]):
        cur = coordinates + tuple(self.dir)
        if cur not in self.position_visits:
            self.position_visits.add(cur)
            return False
        return True


class AdventDay6:
    def __init__(self):
        self.room_map = {}
        with open("puzzleInput.txt") as f:
            for i, line in enumerate(f.readlines()):
                for j, char in enumerate(line):
                    if char == "\n":
                        continue
                    if char == "^":
                        self.guard = Guard([i, j])
                        char = "S"
                    self.room_map[(i, j)] = char
        self.part1 = 0
        self.part2 = 0

    def get_patrol_route(self, part_1: bool = True):
        while self.guard.in_bounds:
            try:
                next_tile = self.room_map[tuple(self.guard.find_next_move())]
                if next_tile != "#":
                    loop = self.guard.accept_move(next_tile, part_1)
                    self.room_map[tuple(self.guard.position)] = "!"
                    if not part_1 and loop:
                        self.part2 += 1
                        break
                else:
                    self.guard.change_dir()
            except KeyError:
                self.guard.in_bounds = False
        if part_1:
            self.part1 = self.guard.positions_seen

    def find_potential_loops(self):
        for obstical_potential in self.guard.main_path:
            self.room_map[obstical_potential] = "#"
            self.guard.restart_loop_tracker()
            self._reset_map()
            self.get_patrol_route(part_1=False)
            self.room_map[obstical_potential] = "."

    def _reset_map(self):
        for key, val in self.room_map.items():
            if val == "!":
                self.room_map[key] = "."


if __name__ == "__main__":
    day6 = AdventDay6()
    day6.get_patrol_route()
    day6.find_potential_loops()
    print(day6.part1, day6.part2)

5080 1919


# --- Part Two ---

While The Historians begin working around the guard's patrol route, you borrow their fancy device and step outside the lab. From the safety of a supply closet, you time travel through the last few months and record the nightly status of the lab's guard post on the walls of the closet.

Returning after what seems like only a few seconds to The Historians, they explain that the guard's patrol area is simply too large for them to safely search the lab without getting caught.

Fortunately, they are pretty sure that adding a single new obstruction won't cause a time paradox. They'd like to place the new obstruction in such a way that the guard will get stuck in a loop, making the rest of the lab safe to search.

To have the lowest chance of creating a time paradox, The Historians would like to know all of the possible positions for such an obstruction. The new obstruction can't be placed at the guard's starting position - the guard is there right now and would notice.

In the above example, there are only 6 different positions where a new obstruction would cause the guard to get stuck in a loop. The diagrams of these six situations use O to mark the new obstruction, | to show a position where the guard moves up/down, - to show a position where the guard moves left/right, and + to show a position where the guard moves both up/down and left/right.

Option one, put a printing press next to the guard's starting position:

....#.....
....+---+#
....|...|.
..#.|...|.
....|..#|.
....|...|.
.#.O^---+.
........#.
#.........
......#...

Option two, put a stack of failed suit prototypes in the bottom right quadrant of the mapped area:

....#.....
....+---+#
....|...|.
..#.|...|.
..+-+-+#|.
..|.|.|.|.
.#+-^-+-+.
......O.#.
#.........
......#...

Option three, put a crate of chimney-squeeze prototype fabric next to the standing desk in the bottom right quadrant:

....#.....
....+---+#
....|...|.
..#.|...|.
..+-+-+#|.
..|.|.|.|.
.#+-^-+-+.
.+----+O#.
#+----+...
......#...

Option four, put an alchemical retroencabulator near the bottom left corner:

....#.....
....+---+#
....|...|.
..#.|...|.
..+-+-+#|.
..|.|.|.|.
.#+-^-+-+.
..|...|.#.
#O+---+...
......#...

Option five, put the alchemical retroencabulator a bit to the right instead:

....#.....
....+---+#
....|...|.
..#.|...|.
..+-+-+#|.
..|.|.|.|.
.#+-^-+-+.
....|.|.#.
#..O+-+...
......#...

Option six, put a tank of sovereign glue right next to the tank of universal solvent:

....#.....
....+---+#
....|...|.
..#.|...|.
..+-+-+#|.
..|.|.|.|.
.#+-^-+-+.
.+----++#.
#+----++..
......#O..

It doesn't really matter what you choose to use as an obstacle so long as you and The Historians can put it into position without the guard noticing. The important thing is having enough options that you can find one that minimizes time paradoxes, and in this example, there are 6 different positions you could choose.

You need to get the guard stuck in a loop by adding a single new obstruction. How many different positions could you choose for this obstruction?
